# Motion Autoencoder (Inference)

## Imports

In [1]:
from matplotlib import pyplot as plt

import motion_model
import motion_synthesis
import motion_sender
import motion_gui
import motion_control


import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn
from collections import OrderedDict
import networkx as nx
import scipy.linalg as sclinalg

import os, sys, time, subprocess
import numpy as np
import math
import pickle
import re

from common import utils
from common import bvh_tools as bvh
from common import fbx_tools as fbx
from common import mocap_tools as mocap
from common.quaternion import qmul, qrot, qnormalize_np, qfix
from common.quaternion_np import slerp
from common.pose_renderer import PoseRenderer

import IPython
from IPython.display import display
import ipywidgets as widgets

In [2]:
%gui qt

## Settings

### Compute Device

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


### Mocap Settings

In [4]:
mocap_file_path = "../../../Data/Mocap"
mocap_files = ["Daniel_ChineseRoom_Take1_50fps.fbx", "DeanNoel_Ocean_Take2_15.12.fbx", "Ephraim_Sport_Take2_15.12.fbx", "MagdaPascalAmbience_Take2_15.12.fbx", "SelinHabibaNic_Shopping_Take2_15.12.fbx"]
mocap_pos_scale = 1.0
mocap_fps = 50

mocap_pos_scale_gui = widgets.FloatText(mocap_pos_scale, description="Mocap Position Scale:", style={'description_width': 'initial'})
mocap_fps_gui = widgets.IntText(mocap_fps, description="Mocap FPS:", style={'description_width': 'initial'})

mocap_files_all = [f for f in os.listdir(mocap_file_path) if os.path.isfile(os.path.join(mocap_file_path, f))]
#print(mocap_files_all)
mocap_files_gui = widgets.SelectMultiple(
    options=mocap_files_all,
    value=mocap_files,  # default: first option selected; can be empty
    description='Mocap Files:',
    layout=widgets.Layout(width='400px'),
    style={'description_width': 'initial'}
)

display(mocap_pos_scale_gui)
display(mocap_fps_gui)
display(mocap_files_gui)

FloatText(value=1.0, description='Mocap Position Scale:', style=DescriptionStyle(description_width='initial'))

IntText(value=50, description='Mocap FPS:', style=DescriptionStyle(description_width='initial'))

SelectMultiple(description='Mocap Files:', index=(0, 2, 4, 6, 8), layout=Layout(width='400px'), options=('Dani…

In [5]:
mocap_pos_scale = mocap_pos_scale_gui.value
mocap_fps = mocap_fps_gui.value
mocap_files = mocap_files_gui.value

### Model Settings

In [6]:
latent_dim = 32
sequence_length = 64
ae_rnn_layer_count = 2
ae_rnn_layer_size = 512
ae_dense_layer_sizes = [ 512 ]

ae_encoder_weights_file = "../../../Data/Models/MotionTransformation/All/weights/encoder_weights_epoch_600"
ae_decoder_weights_file = "../../../Data/Models/MotionTransformation/All/weights/decoder_weights_epoch_600"

latent_dim_gui = widgets.IntText(value=latent_dim, description="Latent Dimension:", style={'description_width': 'initial'})
sequence_length_gui = widgets.IntText(value=sequence_length, description="Sequence Length (Frames):", style={'description_width': 'initial'})
ae_rnn_layer_count_gui = widgets.IntText(value=ae_rnn_layer_count, description="LSTM Layer Count:", style={'description_width': 'initial'})
ae_rnn_layer_size_gui = widgets.IntText(value=ae_rnn_layer_size, description="LSTM Layer Size:", style={'description_width': 'initial'})

ae_dense_layer_sizes_gui = widgets.Textarea(
    value=','.join(list(map(str, ae_dense_layer_sizes))),
    placeholder='Enter dense layer sizes separated by commas',
    description='Dense Layer Sizes:',
    layout=widgets.Layout(width='50%'),
    style={'description_width': 'initial'}
)

ae_encoder_weights_file_gui = widgets.Text(value=ae_encoder_weights_file, description="Encoder Weights File:", style={'description_width': 'initial'}) 
ae_decoder_weights_file_gui = widgets.Text(value=ae_decoder_weights_file, description="Decoder Weights File:", style={'description_width': 'initial'}) 

display(latent_dim_gui)
display(sequence_length_gui)
display(ae_rnn_layer_count_gui)
display(ae_rnn_layer_size_gui)
display(ae_dense_layer_sizes_gui)
display(ae_encoder_weights_file_gui)
display(ae_decoder_weights_file_gui)

IntText(value=32, description='Latent Dimension:', style=DescriptionStyle(description_width='initial'))

IntText(value=64, description='Sequence Length (Frames):', style=DescriptionStyle(description_width='initial')…

IntText(value=2, description='LSTM Layer Count:', style=DescriptionStyle(description_width='initial'))

IntText(value=512, description='LSTM Layer Size:', style=DescriptionStyle(description_width='initial'))

Textarea(value='512', description='Dense Layer Sizes:', layout=Layout(width='50%'), placeholder='Enter dense l…

Text(value='../../../Data/Models/MotionTransformation/All/weights/encoder_weights_epoch_600', description='Enc…

Text(value='../../../Data/Models/MotionTransformation/All/weights/decoder_weights_epoch_600', description='Dec…

In [7]:
latent_dim = latent_dim_gui.value
sequence_length = sequence_length_gui.value
ae_rnn_layer_count = ae_rnn_layer_count_gui.value
ae_rnn_layer_size = ae_rnn_layer_size_gui.value
ae_dense_layer_sizes  = [int(s) for s in re.split(r"\s*,\s*", ae_dense_layer_sizes_gui.value) if s.strip()]
ae_encoder_weights_file = ae_encoder_weights_file_gui.value
ae_decoder_weights_file = ae_decoder_weights_file_gui.value

### OSC Settings

#### OSC Receive Settings

In [8]:
osc_receive_ip = "0.0.0.0"
osc_receive_port = 9002

osc_receive_ip_gui = widgets.Text(value=osc_receive_ip, description="OSC Receive IP:", style={'description_width': 'initial'}) 
osc_receive_port_gui = widgets.IntText(value=osc_receive_port, description="OSC Receive Port:", style={'description_width': 'initial'})

display(osc_receive_ip_gui)
display(osc_receive_port_gui)

Text(value='0.0.0.0', description='OSC Receive IP:', style=TextStyle(description_width='initial'))

IntText(value=9002, description='OSC Receive Port:', style=DescriptionStyle(description_width='initial'))

In [9]:
osc_receive_ip = osc_receive_ip_gui.value
osc_receive_port = osc_receive_port_gui.value

#### OSC Send Settings

In [10]:
osc_send_ip = "127.0.0.1"
osc_send_port = 9004

osc_send_ip_gui = widgets.Text(value=osc_send_ip, description="OSC Send IP:", style={'description_width': 'initial'}) 
osc_send_port_gui = widgets.IntText(value=osc_send_port, description="OSC Send Port:", style={'description_width': 'initial'})

display(osc_send_ip_gui)
display(osc_send_port_gui)

Text(value='127.0.0.1', description='OSC Send IP:', style=TextStyle(description_width='initial'))

IntText(value=9004, description='OSC Send Port:', style=DescriptionStyle(description_width='initial'))

In [11]:
osc_send_ip = osc_send_ip_gui.value
osc_send_port = osc_send_port_gui.value

## Load Mocap Data

In [12]:
bvh_tools = bvh.BVH_Tools()
fbx_tools = fbx.FBX_Tools()
mocap_tools = mocap.Mocap_Tools()

all_mocap_data = []

for mocap_file in mocap_files:
    
    print("process file ", mocap_file)
    
    if mocap_file.endswith(".bvh") or mocap_file.endswith(".BVH"):
        bvh_data = bvh_tools.load(mocap_file_path + "/" + mocap_file)
        mocap_data = mocap_tools.bvh_to_mocap(bvh_data)
    elif mocap_file.endswith(".fbx") or mocap_file.endswith(".FBX"):
        fbx_data = fbx_tools.load(mocap_file_path + "/" + mocap_file)
        mocap_data = mocap_tools.fbx_to_mocap(fbx_data)[0] # first skeleton only
    
    mocap_data["skeleton"]["offsets"] *= mocap_pos_scale
    mocap_data["motion"]["pos_local"] *= mocap_pos_scale
    
    # set x and z offset of root joint to zero
    mocap_data["skeleton"]["offsets"][0, 0] = 0.0 
    mocap_data["skeleton"]["offsets"][0, 2] = 0.0 

    if mocap_file.endswith(".bvh") or mocap_file.endswith(".BVH"):
        mocap_data["motion"]["rot_local"] = mocap_tools.euler_to_quat_bvh(mocap_data["motion"]["rot_local_euler"], mocap_data["rot_sequence"])
    elif mocap_file.endswith(".fbx") or mocap_file.endswith(".FBX"):
        mocap_data["motion"]["rot_local"] = mocap_tools.euler_to_quat(mocap_data["motion"]["rot_local_euler"], mocap_data["rot_sequence"])

    all_mocap_data.append(mocap_data)


all_pose_sequences = []

for mocap_data in all_mocap_data:
    
    pose_sequence = mocap_data["motion"]["rot_local"].astype(np.float32)
    all_pose_sequences.append(pose_sequence)

joint_count = all_pose_sequences[0].shape[1]
joint_dim = all_pose_sequences[0].shape[2]
pose_dim = joint_count * joint_dim

process file  Daniel_ChineseRoom_Take1_50fps.fbx
process file  DeanNoel_Ocean_Take2_15.12.fbx
process file  Ephraim_Sport_Take2_15.12.fbx
process file  MagdaPascalAmbience_Take2_15.12.fbx
process file  SelinHabibaNic_Shopping_Take2_15.12.fbx


## Load Model

In [13]:
motion_model.config["seq_length"] = sequence_length
motion_model.config["data_dim"] = pose_dim
motion_model.config["latent_dim"] = latent_dim
motion_model.config["rnn_layer_count"] = ae_rnn_layer_count
motion_model.config["rnn_layer_size"] = ae_rnn_layer_size
motion_model.config["dense_layer_sizes"] = ae_dense_layer_sizes
motion_model.config["device"] = device
motion_model.config["weights_path"] = [ae_encoder_weights_file, ae_decoder_weights_file]

encoder, decoder = motion_model.createModels(motion_model.config) 

C:\Users\dbisig\Teaching\Offenburg_2025\Software\MotionTransformation\Inference_Reconstruction\motion_model.py:159: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  encoder.loa

## Setup Motion Synthesis

In [14]:
sequence_overlap = sequence_length // 4 * 3

motion_synthesis.config["skeleton"] = all_mocap_data[0]["skeleton"]
motion_synthesis.config["model_encoder"] = encoder
motion_synthesis.config["model_decoder"] = decoder
motion_synthesis.config["device"] = device
motion_synthesis.config["seq_window_length"] = sequence_length
motion_synthesis.config["seq_window_overlap"] = sequence_overlap
motion_synthesis.config["orig_sequences"] = all_pose_sequences
motion_synthesis.config["orig_seq1_index"] = 0
motion_synthesis.config["orig_seq2_index"] = 0

synthesis = motion_synthesis.MotionSynthesis(motion_synthesis.config)

## Create OSC Sender

In [15]:
motion_sender.config["ip"] = osc_send_ip
motion_sender.config["port"] = osc_send_port

osc_sender = motion_sender.OscSender(motion_sender.config)

## Create Application

In [16]:
from PyQt5 import QtWidgets
from PyQt5.QtCore import Qt
import pyqtgraph as pg
import pyqtgraph.opengl as gl
from pathlib import Path

motion_gui.config["synthesis"] = synthesis
motion_gui.config["sender"] = osc_sender

app = QtWidgets.QApplication(sys.argv)
gui = motion_gui.MotionGui(motion_gui.config)

# set close event
def closeEvent():
    QtWidgets.QApplication.quit()
app.lastWindowClosed.connect(closeEvent) # myExitHandler is a callable

## Create OSC Control

In [17]:
motion_control.config["motion_seq"] = pose_sequence
motion_control.config["synthesis"] = synthesis
motion_control.config["gui"] = gui
motion_control.config["latent_dim"] = latent_dim
motion_control.config["ip"] = osc_receive_ip
motion_control.config["port"] = osc_receive_port

osc_control = motion_control.MotionControl(motion_control.config)

## Start Application

In [18]:
osc_control.start()
gui.show()
#app.exec_()

## Interactive Control

In [19]:
motion_source_index_1_gui = widgets.IntText(value=synthesis.orig_seq1_index, description="Motion File 1 Index:", style={'description_width': 'initial'})
motion_source_index_2_gui = widgets.IntText(value=synthesis.orig_seq2_index, description="Motion File 2 Index:", style={'description_width': 'initial'})

motion_source_frame_range_1_gui = widgets.IntRangeSlider(
    value=[synthesis.orig_seq1_frame_range[0],  synthesis.orig_seq1_frame_range[1]],
    min=synthesis.seq_window_length,
    max=synthesis.seq1_length - synthesis.seq_window_length,
    description='Motion File 1 Frame Range:',
    readout=True,
    orientation='horizontal',  # can also be 'vertical'
    layout=widgets.Layout(width='90%'),
    style={'description_width': 'initial'}
)

motion_source_frame_range_2_gui = widgets.IntRangeSlider(
    value=[synthesis.orig_seq2_frame_range[0],  synthesis.orig_seq2_frame_range[1]],
    min=synthesis.seq_window_length,
    max=synthesis.seq2_length - synthesis.seq_window_length,
    description='Motion File 2 Frame Range:',
    readout=True,
    orientation='horizontal',  # can also be 'vertical'
    layout=widgets.Layout(width='90%'),
    style={'description_width': 'initial'}
)

motion_frame_incr_1_gui = widgets.IntText(value=synthesis.orig_seq1_frame_incr, description="Motion File 1 Frame Increment:", style={'description_width': 'initial'})
motion_frame_incr_2_gui = widgets.IntText(value=synthesis.orig_seq2_frame_incr, description="Motion File 2 Frame Increment:", style={'description_width': 'initial'})

motion_encoding_mix_factors = torch.zeros((latent_dim), dtype=torch.float32).to(device)
motion_encoding_offset_factors = torch.zeros((latent_dim), dtype=torch.float32).to(device)

motion_encoding_mix_factor_gui = []
motion_encoding_offset_factor_gui = []

motion_encoding_mix_factors_gui = widgets.FloatSlider(
    value=0.0,
    min=-4.0,
    max=4.0,
    step=0.01,
    description='Mix Factors All:',
    readout_format='.1f',
    layout=widgets.Layout(width='90%'),
    style={'description_width': 'initial'}
)

motion_encoding_offset_factors_gui = widgets.FloatSlider(
    value=0.0,
    min=-4.0,
    max=4.0,
    step=0.01,
    description='Mix Factors All:',
    readout_format='.1f',
    layout=widgets.Layout(width='90%'),
    style={'description_width': 'initial'}
)

for ld in range(latent_dim):

    mix_factor_gui = widgets.FloatSlider(
        value=0.0,
        min=-4.0,
        max=4.0,
        step=0.01,
        description='Mix Factor Dim {}:'.format(ld),
        readout_format='.1f',
        layout=widgets.Layout(width='90%'),
        style={'description_width': 'initial'}
    )

    offset_factor_gui = widgets.FloatSlider(
        value=0.0,
        min=-4.0,
        max=4.0,
        step=0.01,
        description='Offset Factor Dim {}:'.format(ld),
        readout_format='.1f',
        layout=widgets.Layout(width='90%'),
        style={'description_width': 'initial'}
    )

    motion_encoding_mix_factor_gui.append(mix_factor_gui)
    motion_encoding_offset_factor_gui.append(offset_factor_gui)


display(motion_source_index_1_gui)
display(motion_source_frame_range_1_gui)
display(motion_frame_incr_1_gui)

display(motion_source_index_2_gui)
display(motion_source_frame_range_2_gui)
display(motion_frame_incr_2_gui)

display(motion_encoding_mix_factors_gui)

for ld in range(latent_dim):
    display(motion_encoding_mix_factor_gui[ld])

display(motion_encoding_offset_factors_gui)

for ld in range(latent_dim):
    display(motion_encoding_offset_factor_gui[ld])

def on_motion_source_index_1_change(value):
    
    seq_index = value['new']
    
    synthesis.setSeq1Index(seq_index)

def on_motion_source_index_2_change(value):
    
    seq_index = value['new']
    
    synthesis.setSeq2Index(seq_index)

def on_motion_source_frame_range_1_change(value):

    range_start = value['new'][0]
    range_end = value['new'][1]

    synthesis.setSeq1FrameRange(range_start, range_end)

def on_motion_source_frame_range_2_change(value):

    range_start = value['new'][0]
    range_end = value['new'][1]

    synthesis.setSeq2FrameRange(range_start, range_end)

def on_motion_frame_incr_1_change(value):
    
    frame_incr = value['new']
    
    synthesis.setSeq1FrameIncrement(frame_incr)

def on_motion_frame_incr_2_change(value):
    
    frame_incr = value['new']
    
    synthesis.setSeq2FrameIncrement(frame_incr)

def on_motion_encoding_mix_factors_change(value):
    global motion_encoding_mix_factors

    mix_factor = value['new']

    for ld in range(latent_dim):
        motion_encoding_mix_factors[ld] = mix_factor
        motion_encoding_mix_factor_gui[ld].value = mix_factor

    synthesis.setEncodingMix(motion_encoding_mix_factors)

def on_motion_encoding_offset_factors_change(value):
    global motion_encoding_offset_factors

    offset_factor = value['new']

    for ld in range(latent_dim):
        motion_encoding_offset_factors[ld] = offset_factor
        motion_encoding_offset_factor_gui[ld].value = offset_factor

    synthesis.setEncodingOffset(motion_encoding_offset_factors)
    
def on_motion_encoding_mix_factor_change(value):
    global motion_encoding_mix_factors
    
    encoding_mix_gui = value['owner']
    ld = motion_encoding_mix_factor_gui.index(encoding_mix_gui)
    motion_encoding_mix_factors[ld] = value['new']

    synthesis.setEncodingMix(motion_encoding_mix_factors)

def on_motion_encoding_offset_factor_change(value):
    global motion_encoding_offset_factors

    encoding_offset_gui = value['owner']
    ld = motion_encoding_offset_factor_gui.index(encoding_offset_gui)
    motion_encoding_offset_factors[ld] = value['new']

    synthesis.setEncodingOffset(motion_encoding_offset_factors)

motion_source_index_1_gui.observe(on_motion_source_index_1_change, names='value')
motion_source_index_2_gui.observe(on_motion_source_index_2_change, names='value')
motion_source_frame_range_1_gui.observe(on_motion_source_frame_range_1_change, names='value')
motion_source_frame_range_2_gui.observe(on_motion_source_frame_range_2_change, names='value')
motion_frame_incr_1_gui.observe(on_motion_frame_incr_1_change, names='value')
motion_frame_incr_2_gui.observe(on_motion_frame_incr_2_change, names='value')

motion_encoding_mix_factors_gui.observe(on_motion_encoding_mix_factors_change, names='value')
motion_encoding_offset_factors_gui.observe(on_motion_encoding_offset_factors_change, names='value')

for ld in range(latent_dim):
    motion_encoding_mix_factor_gui[ld].observe(on_motion_encoding_mix_factor_change, names='value')
    motion_encoding_offset_factor_gui[ld].observe(on_motion_encoding_offset_factor_change, names='value')

IntText(value=0, description='Motion File 1 Index:', style=DescriptionStyle(description_width='initial'))

IntRangeSlider(value=(64, 24698), description='Motion File 1 Frame Range:', layout=Layout(width='90%'), max=24…

IntText(value=16, description='Motion File 1 Frame Increment:', style=DescriptionStyle(description_width='init…

IntText(value=0, description='Motion File 2 Index:', style=DescriptionStyle(description_width='initial'))

IntRangeSlider(value=(64, 24698), description='Motion File 2 Frame Range:', layout=Layout(width='90%'), max=24…

...

FloatSlider(value=0.0, description='Offset Factor Dim 27:', layout=Layout(width='90%'), max=4.0, min=-4.0, rea…

FloatSlider(value=0.0, description='Offset Factor Dim 28:', layout=Layout(width='90%'), max=4.0, min=-4.0, rea…

FloatSlider(value=0.0, description='Offset Factor Dim 29:', layout=Layout(width='90%'), max=4.0, min=-4.0, rea…

FloatSlider(value=0.0, description='Offset Factor Dim 30:', layout=Layout(width='90%'), max=4.0, min=-4.0, rea…

FloatSlider(value=0.0, description='Offset Factor Dim 31:', layout=Layout(width='90%'), max=4.0, min=-4.0, rea…

## Stop OSC Control

In [ ]:
#osc_control.stop()